In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import json
import time
from lscn import LSCNsClassifier

In [2]:
TRAIN_EN_X = "../memory/enfr/en/train_x.npy"
TRAIN_EN_Y = "../memory/enfr/en/train_y.npy"
VALID_EN_X = "../memory/enfr/en/valid_x.npy"
VALID_EN_Y = "../memory/enfr/en/valid_y.npy"

In [3]:
def load_phone_idx(file_path="../memory/enfr/en/phone_idx.json"):
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

In [4]:
train_x = np.load(TRAIN_EN_X, allow_pickle=True)
train_y = np.load(TRAIN_EN_Y, allow_pickle=True)
valid_x = np.load(VALID_EN_X, allow_pickle=True)
valid_y = np.load(VALID_EN_Y, allow_pickle=True)
phone2idx = load_phone_idx()

### labels are [0, 1, 2, 3, 4, 5, 8], change 8 to 6

In [5]:
train_y[train_y == 8] = len(np.unique(train_y)) - 1
valid_y[valid_y == 8] = len(np.unique(valid_y)) - 1

In [6]:
np.unique(train_y)

array([0, 1, 2, 3, 4, 5, 6])

In [7]:
class PhoneDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y
        self.length = len(Y)
    
    def __len__(self):
        return self.length
    
    def __getitem__(self, i):
        return self.X[i], self.Y[i]
    
    def collate_fn(batch):
        batch_x = [torch.as_tensor(x) for x, y in batch]
        batch_x_length = [len(x) for x in batch]
        batch_x_padded = pad_sequence(batch_x, batch_first=True)
        batch_y = torch.as_tensor([y for x, y in batch])
        return batch_x_padded, batch_x_length, batch_y

In [8]:
def train_epoch(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    total_predictions = 0
    correct_predictions = 0
    
    start_time = time.time()
    for batch_idx, (x_padded, x_lengths, target) in enumerate(train_loader):
        optimizer.zero_grad()
        x_padded = x_padded.to(device)
        target = target.to(device)
        outputs = model(x_padded, x_lengths)
        
        loss = criterion(outputs, target)
        _, predicted = torch.max(outputs.data, 1)
        total_predictions += target.size(0)
        correct_predictions += (predicted == target).sum().item()
        running_loss += loss.item()
        
        loss.backward()
        optimizer.step()
    
    end_time = time.time()
    running_loss /= len(train_loader)
    accuracy = (correct_predictions/total_predictions) * 100.0
    print("Training loss: ", running_loss, "Time: ", end_time - start_time, 's')
    print("Training Accuracy", accuracy, "%")
    return running_loss, accuracy

In [9]:
def valid_epoch(model, valid_loader, criterion, device):
    start_time = time.time()
    with torch.no_grad():
        model.eval()
        running_loss = 0.0
        total_predictions = 0
        correct_predictions = 0
    
        for batch_idx, (x_padded, x_lengths, target) in enumerate(train_loader):
            x_padded = x_padded.to(device)
            target = target.to(device)
            outputs = model(x_padded, x_lengths)
            _, predicted = torch.max(outputs.data, 1)
            total_predictions += target.size(0)
            correct_predictions += (predicted == target).sum().item()
            loss = criterion(outputs, target).detach()
            running_loss += loss.item()
    
    end_time = time.time()
    running_loss /= len(train_loader)
    accuracy = (correct_predictions/total_predictions) * 100.0
    print("Validation loss: ", running_loss, "Time: ", end_time - start_time, 's')
    print("Validation Accuracy", accuracy, "%")
    return running_loss, accuracy

In [10]:
train_dataset = PhoneDataset(train_x, train_y)
valid_dataset = PhoneDataset(valid_x, valid_y)

In [11]:
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=32, num_workers=8, collate_fn=PhoneDataset.collate_fn)
valid_loader = DataLoader(valid_dataset, shuffle=False, batch_size=32, collate_fn=PhoneDataset.collate_fn)

In [12]:
vocab_size =  len(phone2idx)
num_classes = len(np.unique(train_y))

In [13]:
model = LSCNsClassifier(vocab_size, num_classes)

In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
model.to(device)
print(model)

LSCNsClassifier(
  (embed): Embedding(121, 128)
  (convA): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
  (convB): Conv1d(128, 128, kernel_size=(5,), stride=(1,), padding=(2,))
  (lstm): LSTM(256, 128, batch_first=True)
  (linear): Linear(in_features=128, out_features=7, bias=True)
)


In [15]:
num_epochs = 120

In [ ]:
for epoch in range(1, num_epochs + 1):
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    valid_loss, valid_acc = valid_epoch(model, valid_loader, criterion, device)
    print("Epoch {} finished.".format(epoch))
    print('='*20)